<a href="https://colab.research.google.com/github/Sirath07/projects-emotion_detector.ipynb/blob/main/emotion_streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.3 MB/s eta 0:00:00


In [3]:
# streamlit_app.py
import streamlit as st
import torch
import numpy as np
from PIL import Image
import io
import cv2
import torch.nn.functional as F
from torchvision import transforms
import os

# --- Config ---
MODEL_PATH = "best_model.pth"   # name of model file in repo
EMOTIONS = ["Angry","Disgust","Fear","Happy","Sad","Surprise","Neutral"]

st.set_page_config(page_title="Face Emotion Recognition", page_icon="😊")

st.title("Face Emotion Recognition (PyTorch)")
st.write("Upload a face image and the model will predict the emotion.")

# --- Load model (lazy loading) ---
@st.cache_resource(show_spinner=False)
def load_model(path):
    # The model architecture must match the saved model
    # Here we assume you used a ResNet18-based model in training (3-channel input)
    import torch.nn as nn
    from torchvision import models

    model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, len(EMOTIONS))
    state = torch.load(path, map_location="cpu")
    model.load_state_dict(state)
    model.eval()
    return model

if not os.path.exists(MODEL_PATH):
    st.error(f"Model file not found in app folder: {MODEL_PATH}. Upload it to the repo.")
else:
    model = load_model(MODEL_PATH)

    uploaded_file = st.file_uploader("Upload an image (jpg / png)", type=["jpg","jpeg","png"])
    if uploaded_file is not None:
        # read image
        image = Image.open(uploaded_file).convert("RGB")
        st.image(image, caption="Uploaded image", use_column_width=True)

        # Convert to numpy and detect face with OpenCV Haarcascade
        img_np = np.array(image)
        gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(30,30))

        if len(faces) == 0:
            st.warning("No face detected. The whole image will be used for prediction.")
            face_img = cv2.resize(gray, (48,48))
        else:
            # use the first detected face
            x,y,w,h = faces[0]
            face_roi = gray[y:y+h, x:x+w]
            face_img = cv2.resize(face_roi, (48,48))

        # preprocess to match training
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
        ])

        # model was trained on 3-channel stacked grayscale; make 3 channels
        face_rgb = np.stack([face_img,face_img,face_img], axis=-1)
        tensor = transform(face_rgb).unsqueeze(0)  # (1,3,48,48)

        with st.spinner("Predicting..."):
            with torch.no_grad():
                outputs = model(tensor)
                probs = F.softmax(outputs, dim=1).cpu().numpy()[0]
                pred_idx = int(np.argmax(probs))

        st.success(f"Predicted emotion: **{EMOTIONS[pred_idx]}**")
        st.write("Probabilities:")
        for i, e in enumerate(EMOTIONS):
            st.write(f"- {e}: {probs[i]:.3f}")


2025-12-07 19:49:01.376 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 19:49:01.378 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 19:49:01.541 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-07 19:49:01.543 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 19:49:01.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 19:49:01.549 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-07 19:49:01.551 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn